In [1]:
import os

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.vectorstores import DeepLake

from dotenv import load_dotenv
load_dotenv()

c:\Users\krish\miniconda3\envs\llmsenv\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.9) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


True

In [2]:
loader = UnstructuredMarkdownLoader(
    file_path="C:/Users/***/ars.md"
)

docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)


embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

#### Creating a vector store on Activeloop

In [ ]:
my_activeloop_org_id = "****"
my_activeloop_dataset_name = "****"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path, embedding=embeddings)

#### Creating a vector store on AWS S3

In [3]:
dataset_path = "s3://BUCKET_NAME/FOLDER_NAME/"

db = DeepLake.from_documents(
    splits,
    dataset_path=dataset_path,
    embedding=embeddings,
    overwrite=True,
    creds={
        "aws_access_key_id": os.getenv("AWS_ACCESS_KEY_ID"),
        "aws_secret_access_key": os.getenv("AWS_SECRET_ACCESS_KEY")
    }
)

Creating 33 embeddings in 1 batches of size 33:: 100%|██████████| 1/1 [00:04<00:00,  4.33s/it]

Dataset(path='s3://arsbot-dataset/prog-locs/', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (33, 1)      str     None   
 metadata     json      (33, 1)      str     None   
 embedding  embedding  (33, 1536)  float32   None   
    id        text      (33, 1)      str     None   


In [4]:
db_test = DeepLake(
    dataset_path=dataset_path,
    read_only=True,
    creds={
        "aws_access_key_id": os.getenv("AWS_ACCESS_KEY_ID"),
        "aws_secret_access_key": os.getenv("AWS_SECRET_ACCESS_KEY")
    }
)

Deep Lake Dataset in s3://arsbot-dataset/prog-locs/ already exists, loading from the storage


In [6]:
db_test.vectorstore.summary()

Dataset(path='s3://arsbot-dataset/prog-locs/', read_only=True, tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (33, 1536)  float32   None   
    id        text      (33, 1)      str     None   
 metadata     json      (33, 1)      str     None   
   text       text      (33, 1)      str     None   


In [8]:
db.similarity_search("Programs offering")

[Document(page_content='Find A Location Near You\n\nInpatient Drug and Alcohol Programs\n\nOur inpatient drug and alcohol treatment allows patients the opportunity to focus on understanding and healing from their addiction while also treating any co-occurring mental health conditions.\n\nInpatient Rehab at Advanced Recovery Systems\n\nOur inpatient rehab programs, also commonly known as residential treatment, offers a highly structured and intensive approach to addiction treatment. Patients live on-site at our facilities to focus on their recovery and receive evidence-based programming from our licensed staff throughout the day.', metadata={'source': 'C:/Users/krish/Desktop/OpTS/LanguageModels/ars_bot/ars.md'}),
 Document(page_content='Speak With Inpatient Admissions\nA Recovery Advocate can answer any questions and help you get started in inpatient rehabilitation.\n\nOur Outpatient Rehab Program\n\nOur outpatient facilities offer ongoing drug and alcohol addiction treatment in additio